# Pré-Processamento - Leitura e Limpeza

## Resumo do alcance dessa etapa

Os dados são importados de diferentes fontes e unidos em uma única base.
A cada variável é dado um nome informativo, a base é limpada de inconsitências e ausências e, por fim, a desigualdade (medida como um índice Gini) é classificada em duas categorias: alto e baixo.

Fonte dados:
https://mapasinterativos.ibge.gov.br/agrocompara/

Fonte Gini:

## Detalhamento da etapa

### Imports

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [3]:
import yaml

with open('/mnt/d/Clientes/ODX/agro/online_book/_local_paths.yml') as f:
    paths = yaml.load(f, Loader=yaml.FullLoader)

GINI_DATA_PATH = paths['GINI_DATA_PATH']
CENSO_AGRO_DATA_PATH = paths['CENSO_AGRO_DATA_PATH']

In [4]:
y = pd.read_excel(GINI_DATA_PATH)
df = gpd.read_file(CENSO_AGRO_DATA_PATH)

In [5]:
df.head()

OBJECTID              MUNICIPIO  Shape_Leng  Shape_Area      V1        V2  \
0         1      Lupionópolis - PR    0.514701    0.010596   173.0   59.7001   
1         2  Águas de Lindóia - SP    0.344094    0.004856    69.0   28.4016   
2         3            Uruará - PA    5.505977    0.882292  3993.0  137.3820   
3         4      São Leopoldo - RS    0.505968    0.009694    65.0   26.7850   
4         5            Vargem - SP    0.535624    0.012407   228.0   31.9352   

        V3         V4         V5        V6  ...        GEO       V32  \
0  3.23121   165.7380   852.3680  124.3040  ...  4113809.0  2.457880   
1  3.42029    36.3902    80.0584  133.4310  ...  3500501.0  2.185360   
2  3.05109  1556.9600  4003.6200  800.7230  ...  1508159.0  0.947595   
3  3.44615    85.3800    85.3800   18.9733  ...  4318705.0  1.906410   
4  3.11404    52.0186   127.1570   54.4957  ...  3556354.0  1.693530   

        V33       V34       V35       V36       V37      V38      V39  \
0  1.156070  48.55490  8.586240   8.09249  68.78610  71.0983  36.9942   
1  5.797100  39.13040  2.435280  13.04350  49.27540  60.8696  46.3768   
2  0.075132  30.12770  3.377250   2.17881   5.76008  84.0471  64.1122   
3  1.538460   9.23077  0.754856  20.00000  26.15380  53.8462  40.0000   
4  0.877193  12.71930  2.231690  14.03510  55.26320  54.8246  51.7544   

                                            geometry  
0  POLYGON Z ((-51.63021 -22.66372 0.00000, -51.6...  
1  POLYGON Z ((-46.61019 -22.43936 0.00000, -46.6...  
2  POLYGON Z ((-53.85954 -2.88750 0.00000, -53.85...  
3  POLYGON Z ((-51.17861 -29.69309 0.00000, -51.1...  
4  POLYGON Z ((-46.39678 -22.81069 0.00000, -46.3...  

[5 rows x 45 columns]

In [6]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 45 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    5570 non-null   int64   
 1   MUNICIPIO   5570 non-null   object  
 2   Shape_Leng  5570 non-null   float64 
 3   Shape_Area  5570 non-null   float64 
 4   V1          5570 non-null   float64 
 5   V2          5570 non-null   float64 
 6   V3          5570 non-null   float64 
 7   V4          5570 non-null   float64 
 8   V5          5570 non-null   float64 
 9   V6          5570 non-null   float64 
 10  V7          5570 non-null   float64 
 11  V8          5570 non-null   float64 
 12  V9          5570 non-null   float64 
 13  V10         5570 non-null   float64 
 14  V11         5570 non-null   float64 
 15  V12         5570 non-null   float64 
 16  V13         5570 non-null   float64 
 17  V14         5570 non-null   float64 
 18  V15         5570 non-null   float64 
 19

O arquivo utilizado veio com as variáveis nomeadas de maneira pouco informativa.

Com auxílio do [dicionário de dados](https://mapasinterativos.ibge.gov.br/agrocompara/dados/def_indicadores.pdf), mudamos os rótulos para que sejam mais informativos.

In [7]:
dict_rename = {
    'V1' :'Estabelecimento Agropecuário (N)',
    'V2' :'Área média (ha)',
    'V3' : 'Pessoal Ocupado / Estabelecimento (Pessoa)',
    'V4' :'Área lavoura/ Adubadeira (ha)',
    'V5' :'Área lavoura/ Colheitadeira(ha)',
    'V6' :'Área lavoura/ Semeadeira(ha)',
    'V7' :'Área lavoura/ Trator(ha)',
    'V8' :'Atividade-Lavoura Temporária (%)',
    'V9' :'Atividade-Lavoura Permanente(%)',
    'V10' :'Atividade-Pecuária(%)',
    'V11' :'Atividade-Horticultura&Floricultura(%)',
    'V12' :'Atividade-Sementes&Mudas(%)',
    'V13' :'Atividade-Produção Florestal(%)',
    'V14' :'Atividade-Pesca(%)',
    'V15' :'Atividade-Aquicultura(%)',
    'V16' :'Uso das terras-Lavoura(%)',
    'V17' :'Uso das terras-Pastagem (%)',
    'V18' :'Aves-Corte (%)',
    'V19' :'Aves-Ovos(%)',
    'V20' :'Bovinos-Corte (%)',
    'V21' :'Bovinos-Leite(%)',
    'V22' :'Rendimento-Arroz (kg/ha)',
    'V23' :'Rendimento-Cana (kg/ha)',
    'V24' :'Rendimento-Mandioca (kg/ha)',
    'V25' :'Rendimento-Milho (kg/ha)',
    'V26' :'Rendimento-Soja (kg/ha)',
    'V27' :'Rendimento-Trigo(kg/ha)',
    'V28' :'Rendimento-Cacau(kg/ha)',
    'V29' :'Rendimento-Café(kg/ha)',
    'V30':'Rendimento-Laranja(kg/ha)',
    'V31':'Rendimento-Uva(kg/ha)',
    'V32':'Carga de Bovinos (n/ha)',
    'V33':'Cisterna (%)',
    'V34':'Utilização de Agrotóxicos (%)',
    'V35':'Despesa com Agrotóxicos (%)',
    'V36':'Uso de irrigação (%)',
    'V37':'Assistência Técnica (%)',
    'V38':'Agricultura familiar (%)',
    'V39':'Produtor com escolaridade até Ensino Fundamental(%)',
}

In [8]:
df = df.rename(columns=dict_rename)

Nem todas as variáveis são informativas para essa análise (como as coordenadas dos shapes, por exemplo).
Abaixo, selecionamos as que não foram julgadas relevantes.

In [9]:
variaveis_relevantes = [
    'GEO', 'MUNICIPIO', 'Estabelecimento Agropecuário (N)',
    'Área média (ha)', 'Pessoal Ocupado / Estabelecimento (Pessoa)',
    'Área lavoura/ Adubadeira (ha)', 'Área lavoura/ Colheitadeira(ha)',
    'Área lavoura/ Semeadeira(ha)', 'Área lavoura/ Trator(ha)',
    'Atividade-Lavoura Temporária (%)', 'Atividade-Lavoura Permanente(%)',
    'Atividade-Pecuária(%)', 'Atividade-Horticultura&Floricultura(%)',
    'Atividade-Sementes&Mudas(%)', 'Atividade-Produção Florestal(%)',
    'Atividade-Pesca(%)', 'Atividade-Aquicultura(%)',
    'Uso das terras-Lavoura(%)', 'Uso das terras-Pastagem (%)',
    'Aves-Corte (%)', 'Aves-Ovos(%)', 'Bovinos-Corte (%)',
    'Bovinos-Leite(%)', 'Rendimento-Arroz (kg/ha)',
    'Rendimento-Cana (kg/ha)', 'Rendimento-Mandioca (kg/ha)',
    'Rendimento-Milho (kg/ha)', 'Rendimento-Soja (kg/ha)',
    'Rendimento-Trigo(kg/ha)', 'Rendimento-Cacau(kg/ha)',
    'Rendimento-Café(kg/ha)', 'Rendimento-Laranja(kg/ha)',
    'Rendimento-Uva(kg/ha)', 'Carga de Bovinos (n/ha)',
    'Cisterna (%)', 'Utilização de Agrotóxicos (%)',
    'Despesa com Agrotóxicos (%)', 'Uso de irrigação (%)',
    'Assistência Técnica (%)', 'Agricultura familiar (%)',
    'Produtor com escolaridade até Ensino Fundamental(%)',
    ]

In [10]:
df = df[variaveis_relevantes]

Com os dados do censo agropecuário melhor formatados, seguimos para processamento dos dados da FONTE

In [11]:
print(y.shape)
y.head()

(5563, 6)


COD_UF   NOME_UF  COD_MUN               NOM_MUN     AREAMED      gini
0      11  RONDONIA  1100015  ALTA FLORESTA DOESTE  129.156672  0.836474
1      11  RONDONIA  1100023             ARIQUEMES   114.15835   0.73279
2      11  RONDONIA  1100031                CABIXI  105.194995  0.744177
3      11  RONDONIA  1100049                CACOAL   58.046611  0.695199
4      11  RONDONIA  1100056            CEREJEIRAS  176.197346  0.827577

In [12]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5563 entries, 0 to 5562
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   COD_UF   5563 non-null   int64 
 1   NOME_UF  5563 non-null   object
 2   COD_MUN  5563 non-null   int64 
 3   NOM_MUN  5563 non-null   object
 4   AREAMED  5563 non-null   object
 5   gini     5563 non-null   object
dtypes: int64(2), object(4)
memory usage: 260.9+ KB


In [13]:
y.gini = pd.to_numeric(y.gini, errors='coerce')

Nem todas as variáveis são relevantes, mas a limpeza será feita após o merge

### Criação das categorias

Gini foi binarizado em 'gini_alto' e 'gini_baixo', critério de corte no 50 percentil

In [14]:
# valor de corte
y.gini.quantile(q=.5)

0.7450668545258983

In [15]:
y.gini = np.select(
    [y['gini'] <= y['gini'].quantile(q=.5)],
    ['gini_baixo'], default='gini_alto'
)

### Merge

In [16]:
df = df.merge(y, left_on='GEO', right_on='COD_MUN')

In [17]:
df.shape

(5563, 47)

In [18]:
df.dropna().shape #inefetivo, o merge foi do tipo 'inner_join', não haviam NAs resultantes

(5563, 47)

Em seguida, separamos os labels dos municípios e as variáveis de interesse.

In [19]:
municipios = df['MUNICIPIO']
X = df.drop(columns=[
    'COD_UF', 'NOME_UF', 'COD_MUN', 'NOM_MUN', 'AREAMED', 'GEO', 'MUNICIPIO'
])

A próxima etapa é escalar e codificar as features conforme necessário.

In [20]:
# editar metadata
# tags ['remove-input']

municipios.to_pickle(paths['municipios'])
X.to_pickle(paths['X'])